In [ ]:
import hypermri
import hypermri.utils.utils_anatomical as ut_anat
import sys

import os
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import pandas as pd
import datetime
from scipy.stats import rayleigh
from scipy.optimize import curve_fit 
import matplotlib
from matplotlib.patches import Rectangle

from hypermri.utils.utils_sv_spectroscopy import Plot_Voxel_on_Anat
import hypermri.utils.utils_anatomical as ut_anat
import hypermri.utils.utils_spectroscopy as ut_spec
import hypermri.utils.utils_fitting as ut_fitting

%load_ext autoreload
%autoreload 2

%matplotlib widget

def get_colors_from_cmap(cmap_name, N):
    cmap = plt.get_cmap(cmap_name)
    colors = cmap(np.linspace(0, 1, N))
    return colors


from matplotlib import rc
rc("font", **{"family": "serif", "serif": ["Computer Modern"]})
rc("text", usetex=True)
matplotlib.rcParams.update({"font.size": 11})
import hypermri
from Template import import_paths

basepath,savepath,publication_path=import_paths()
savepath=os.path.join(savepath,'OMPD_mice/')

# Load Data

In [ ]:
path=''
animal_id=''
date=''
base=os.path.dirname(os.path.dirname(basepath))
scans=hypermri.BrukerDir(os.path.join(base,path),verbose=True,keywords=['PVM_NVoxels'])
coronal=scans[7]
axial=scans[8]
mv_press=scans[19]
mv_press_1H=scans[18]
singlepulse=scans[20]
nvox=mv_press.method['PVM_NVoxels']
# name the positions where we have put PRESS Muscle
vox_names=['Tumor','Tumor','Kidney','Kidney','Muscle','Artery']
locations=vox_names

In [ ]:
fig,ax=plt.subplots(1,figsize=(10,5))
lb=0
lb_spec=np.array([np.abs(np.squeeze(mv_press.single_spec_linebroadening(np.squeeze(mv_press.complex_spec)[:,n],lb))) for n in range(len(vox_names))])

[ax.plot(mv_press.ppm_axis,(lb_spec[n,:]-np.mean(lb_spec[n,0:150]))/np.std(lb_spec[n,0:150])-25*n,label=vox_names[n]) for n in range(len(vox_names))]

ax.set_xlim([185,165])
ax.set_xlabel(r'$\sigma$[ppm]')
box=ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax.legend(loc='center left',bbox_to_anchor=(1,0.5))


# 2. Fit spectra and extract frequencies

In [ ]:
experiment=mv_press

metabs = ['OMPD1', 'OMPD5','OMPD5-2','OMPD5-3']

fit_params = {}

fit_params["zoomfactor"] = 1.5
fit_params["max_t2_s"] = 0.05
fit_params["min_t2_s"] = 0.01
fit_params["range_t2s_s"] = 0.02

# get the indices of the peaks:
fit_params["metabs"] = metabs
fit_params["fit_range_repetitions"] = 1
fit_params["range_freqs_Hz"] = 20
fit_params["cut_off"] = 70
fit_params["niter"] = 1 # number of iterations:
fit_params["npoints"] = 21 # number of tested points per iteration:
fit_params["rep_fitting"] = 11 # number of tested points per iteration:
fit_params["provided_dims"] = ["fid"]
fit_params = ut_fitting.def_fit_params(fit_params=fit_params, data_obj=experiment)

cut_off_spec=np.fft.fftshift(np.fft.fft(experiment.complex_fids[fit_params["cut_off"]:,:],axis=0),axes=(0,))
peak_index_ompd1 = ut_spec.find_npeaks(input_data=np.sum(np.squeeze(cut_off_spec),axis=1),
                              freq_range=fit_params['freq_range_ppm'],
                              npeaks=1,
                            plot=True)

# diff literature and measured:
measured_ompd1_freq = fit_params["freq_range_ppm"][peak_index_ompd1]
ompd1_ref_freq = 171.66

diff_ompd1=measured_ompd1_freq-ompd1_ref_freq


fit_params["metabs_freqs_ppm"] = [ompd1_ref_freq,179.06,178.495,177.56]-diff_ompd1

fit_params = ut_fitting.def_fit_params(fit_params=fit_params, data_obj=experiment)

fit_spectrums, fit_amps, fit_freqs, fit_t2s, fit_stds  = ut_fitting.fit_data_pseudo_inv(input_data=cut_off_spec,
                                                                             data_obj=experiment,
                                                                  fit_params=fit_params,
                                                              use_multiprocessing=True)

fit_freqs_ppm = ut_spec.freq_Hz_to_ppm(freq_Hz=np.squeeze(fit_freqs), hz_axis=fit_params["freq_range_Hz"], ppm_axis=fit_params["freq_range_ppm"], ppm_axis_flipped=False)
fit_stds_ppm = ut_spec.freq_Hz_to_ppm(freq_Hz=np.squeeze(fit_stds), hz_axis=fit_params["freq_range_Hz"], ppm_axis=fit_params["freq_range_ppm"], ppm_axis_flipped=False)


# Determine SNR of OMPD 1 and 5 peaks

In [ ]:
fig,ax=plt.subplots(nvox,2,tight_layout=True,figsize=(6,10))
noise_floor=np.ones((nvox))
for n in range(nvox):
    ax[n,0].plot(np.abs((np.squeeze(cut_off_spec)[:,n]-np.mean(np.abs(np.squeeze(cut_off_spec)[0:150,n])))/np.std(np.abs(np.squeeze(cut_off_spec)[0:150,n]))))
    ax[n,0].plot(np.abs((np.squeeze(cut_off_spec)[:,n]-np.mean(np.abs(np.squeeze(cut_off_spec)[0:150,n])))/np.std(np.abs(np.squeeze(cut_off_spec)[0:150,n])))[0:150])
    
    noise_spec=np.abs(((np.squeeze(cut_off_spec)[:,n]-np.mean(np.abs(np.squeeze(cut_off_spec)[0:150,n])))/np.std(np.abs(np.squeeze(cut_off_spec)[0:150,n]))))[0:150]

    params = rayleigh.fit(noise_spec)
    scale = params[1]

    x = np.linspace(0, max(noise_spec), 100)
    pdf_fitted = rayleigh.pdf(x, loc=0, scale=scale)


    ax[n,1].hist(noise_spec, bins=30, density=True, alpha=0.3, color='C0', edgecolor='black')
    ax[n,1].plot(x, pdf_fitted, 'r-', label=f'Rayleigh fit (scale={scale:.2f})')
    fitted_rayleigh = rayleigh(scale=scale)

    mean = fitted_rayleigh.mean()
    std_dev = fitted_rayleigh.std()
    noise_floor[n]=mean+std_dev
    ax[n,1].set_title(str(mean.round(1))+'±'+str(std_dev.round(1)))


In [ ]:
plt.close('all')
fig,ax=plt.subplots(nvox,2,tight_layout=True,figsize=(7,5*nvox/2))
peak_snrs=np.ones((4,nvox))*np.nan
mean_noise=[np.mean(np.abs(np.squeeze(cut_off_spec)[0:150,n])) for n in range(len(locations))]
std_noise=[np.std(np.abs(np.squeeze(cut_off_spec)[0:150,n])) for n in range(len(locations))]

for voxel in range(nvox):
    ax[voxel,0].plot(mv_press.get_ppm(70),np.abs(np.squeeze(cut_off_spec))[:,voxel])
    ax[voxel,0].plot(mv_press.get_ppm(70),np.sum(np.abs(np.squeeze(fit_spectrums)[:,voxel,:]),axis=1),color='r')
    ax[voxel,1].plot(mv_press.get_ppm(70),np.abs(np.squeeze(cut_off_spec))[:,voxel],color='k',alpha=0.3)

    for peak in range(4):
        max_peak_fit_val=np.max(np.abs(np.squeeze(fit_spectrums)[:,voxel,peak]))
        
        snr=np.round((max_peak_fit_val-mean_noise[voxel])/std_noise[voxel],2)
        peak_snrs[peak,voxel]=snr
        ax[voxel,1].plot(mv_press.get_ppm(70),np.abs(np.squeeze(fit_spectrums)[:,voxel,peak]),label=metabs[peak]+',SNR='+str(snr))
        ax[voxel,1].set_title('Noise='+str(np.round(noise_floor[voxel],0)))
    ax[voxel,0].set_xlim([195,155])
    ax[voxel,1].set_xlim([195,155])
    
    ax[voxel,1].legend()
    ax[voxel,0].set_title(locations[voxel])


In [ ]:
fig,ax=plt.subplots(2,3,tight_layout=True,figsize=(8,5))
for n in range(nvox):
    nx,ny=n//3,n%3
    for m in range(4):
        ax[nx,ny].plot(fit_params['freq_range_ppm'],np.abs(np.squeeze(fit_spectrums)[:,n,m]),linewidth=2)
    ax[nx,ny].plot(fit_params['freq_range_ppm'],np.abs(np.squeeze(cut_off_spec)[:,n]),color='k',alpha=0.5,linewidth=1)
    ax[nx,ny].set_xlim([185,165])
    ax[nx,ny].set_xticks([185,180,175,170,165])
    ax[nx,ny].set_title(locations[n])


# Compute frequency shifts and pH Dataframe

In [ ]:
freq_diff_1=np.abs(np.squeeze(fit_freqs_ppm)[:,1]-np.squeeze(fit_freqs_ppm)[:,0])
freq_diff_2=np.abs(np.squeeze(fit_freqs_ppm)[:,2]-np.squeeze(fit_freqs_ppm)[:,0])
freq_diff_3=np.abs(np.squeeze(fit_freqs_ppm)[:,3]-np.squeeze(fit_freqs_ppm)[:,0])

freq_diff_hz_std_1 = np.abs(np.sqrt(fit_stds[..., 1, 1]**2 +
                           fit_stds[..., 0, 1]**2 ))
freq_diff_hz_std_2= np.abs(np.sqrt(fit_stds[..., 2, 1]**2 +
                           fit_stds[..., 0, 1]**2 ))
freq_diff_hz_std_3= np.abs(np.sqrt(fit_stds[..., 3, 1]**2 +
                           fit_stds[..., 0, 1]**2 ))


err_freq_diff_1 = np.squeeze(ut_spec.freq_Hz_to_ppm(freq_Hz=freq_diff_hz_std_1,
                                   hz_axis  = fit_params["freq_range_Hz"],
                                   ppm_axis = fit_params['freq_range_ppm'],
                                   ppm_centered_at_0=True))

err_freq_diff_2 = np.squeeze(ut_spec.freq_Hz_to_ppm(freq_Hz=freq_diff_hz_std_2,
                                   hz_axis  = fit_params["freq_range_Hz"],
                                   ppm_axis = fit_params['freq_range_ppm'],
                                   ppm_centered_at_0=True))
err_freq_diff_3 = np.squeeze(ut_spec.freq_Hz_to_ppm(freq_Hz=freq_diff_hz_std_3,
                                   hz_axis  = fit_params["freq_range_Hz"],
                                   ppm_axis = fit_params['freq_range_ppm'],
                                   ppm_centered_at_0=True))

ph_vals_1,ph_errors_1=ut_fitting.get_pH_from_OMPD(freq_diff_1,err_freq_diff_2)
ph_vals_2,ph_errors_2=ut_fitting.get_pH_from_OMPD(freq_diff_2,err_freq_diff_2)
ph_vals_3,ph_errors_3=ut_fitting.get_pH_from_OMPD(freq_diff_3,err_freq_diff_3)

output_df=pd.DataFrame()
output_df['Voxel']=locations
output_df['pH1']=ph_vals_1
output_df['dpH1']=ph_errors_1
output_df['pH2']=ph_vals_2
output_df['dpH2']=ph_errors_2
output_df['pH3']=ph_vals_3
output_df['dpH3']=ph_errors_3


output_df['SNR OMPD1']=peak_snrs[0,:]
output_df['SNR OMPD5']=peak_snrs[1,:]
output_df['SNR OMPD5-2']=peak_snrs[2,:]
output_df['SNR OMPD5-3']=peak_snrs[3,:]

output_df['noise']=noise_floor
animal_list=[]
animal_num=[]
for n in range(len(locations)):
    animal_list.append(animal_id)
    animal_num.append(2)
output_df['animal']=animal_list
output_df['num']=animal_num
output_df2=output_df
output_df.round(2)

# Anatomicals

In [ ]:
plt.close('all')
fig,ax=plt.subplots(2,len(vox_names),figsize=(2*len(vox_names),2*len(vox_names)))
matplotlib.rcParams.update({"font.size": 11})
for n in range(len(vox_names)):
    Plot_Voxel_on_Anat(mv_press,coronal,ax[0,n],n,0,235,vox_color='C'+str(n))
    ax[0,n].axis('off')
    ax[0,n].set_title(None)
    ax[0,n].set_xlim([12,-12])
    ax[0,n].add_patch(Rectangle((-11,-23),5,1,fc='w'))
    ax[0,n].text(-5.5,-21.5,'5 mm',color='w')
    ax[0,n].text(3,21.5,vox_names[n],color='w',size='11')

for n in range(len(vox_names)):
    Plot_Voxel_on_Anat(mv_press,axial,ax[1,n],n,0,235,vox_color='C'+str(n))
    ax[1,n].axis('off')
    ax[1,n].set_title(None)
    ax[1,n].set_xlim([12,-12])
    ax[1,n].set_ylim([-13,13])
    
    ax[1,n].add_patch(Rectangle((-11,-12),5,1,fc='w'))


plt.subplots_adjust(hspace=0,wspace=0.1)
